In [1]:
# Quick hack to load local SDK code
import os
os.getcwd()
os.chdir("/Users/andres/code/validmind-sdk")

In [2]:
# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import validmind as vm
from validmind import Model, ModelAttributes

vm.init(project="cl2r3k1ri000009jweny7ba1g")

True

In [4]:
model = Model(
    attributes=ModelAttributes(
        architecture="Extreme Gradient Boosting",
        framework="XGBoost",
        framework_version="1.5.2",
    ),
    task="classification",
    subtask="binary",
)

In [5]:
vm.log_model(model)

Model(attributes={'architecture': 'Extreme Gradient Boosting', 'framework': 'XGBoost', 'framework_version': '1.5.2'}, task='classification', subtask='binary', params=None, model_id='main')